# Quotes scraping

in this code nootbook i'll be scraping quotes from goodreads based on the type \
i'll store them in a list of dictionnaries so it would be easy for me to stor them as csv

In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [6]:
#get types of quotes
page = 1
maxPages = 100
url = f'https://www.goodreads.com/quotes/tag/love?page={page}'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
# a list contains the tpes of quotes
types = []
links = soup.select('.listTagsTwoColumn a.gr-hyperlink')
for link in links:
    quoteType = link.text.strip()
    quoteLink = '-'.join(quoteType.split()[:-1])
    types.append( { 'type':quoteType, 'link':quoteLink.lower() } )

In [3]:
# transform the types to dataFrame
tDf = pd.DataFrame(types)
tDf.head(2)
tDf.isna().sum()

type    0
link    0
dtype: int64

In [4]:
# make a GET request to the HTML page
# this code will scrap 30*2000 = 60000 quotes
maxPages = 100
quotes = []
breakIt = False
for quoteType in tDf.values:
    for i in range(1,maxPages+1):
        page = requests.get(f'https://www.goodreads.com/quotes/tag/{quoteType[1]}/?page={i}')
        # create a BeautifulSoup object from the page content
        soup = BeautifulSoup(page.content, 'html.parser')

        # find all the card elements on the page
        cards = soup.find_all('div', class_='quote')
        # loop through each card element and extract its data
        for card in cards:
            title = re.sub('[^A-Za-z0-9]+', ' ', card.find('div', class_='quoteText').text.split('―')[0])
            author = re.sub('[^A-Za-z0-9]+', ' ', card.find('span', class_='authorOrTitle').text)
            try:
                book = re.sub('[^A-Za-z0-9]+', ' ', card.find('a', class_='authorOrTitle').text)
            except:
                book = None
            tags = []
            for t in card.select('.greyText a'):
                tags.append(t.text)
            # add the card data to the array
            quotes.append({
                'title': title,
                'author': author,
                'tags': tags,
                'book' : book,
                'type': quoteType[0],
            })
        if(len(quotes)%100==0):
            print(quoteType[0])
    print(quoteType[0],len(quotes))

quotesDf = pd.DataFrame(quotes)
quotesDf.to_csv('out/quotes.csv',encoding='utf-8')

Love Quotes
Love Quotes 300


In [78]:
# reading the saved data
df = pd.read_csv('out/quotes.csv')

In [87]:
# spliting and labeling data
types = df['type'].unique()
for t in types:
    dDf = df.loc[df['type'] == t ]
    dDf.drop(columns='type')
    dDf.to_csv(f'out/{"-".join(t.split())}.csv',encoding='utf-8')
    